In [1]:
#!/usr/bin/env python
import asyncio
import pyppeteer as ptr
from typing import Optional
import pandas as pd
from glob import glob
import nltk
from nltk.corpus import stopwords
import re
from typing import Callable

In [2]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/urmzd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/urmzd/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/urmzd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
PosTag = tuple[str, str]
PosTagList = list[PosTag]

In [5]:
## Scraper.
async def download_html(browser: ptr.browser.Browser, url: str, selector: str) -> Optional[str]:
    page = await browser.newPage()
    await page.goto(url, waitUntil="load")
    content = await page.querySelector(selector)

    html = ''
    if content:
        html = await page.evaluate('(element) => element.textContent', content)

    return html

def write_to_resource_target(file_name, content: PosTagList):
    with open(file_name, "w") as file:
        file.write(",".join(content))


async def get_training_data_from_folder(folder_path: str) -> None:
    browser = await ptr.launch(headless=True)
    files = glob(folder_path + '/**/*.csv', recursive=True)
    
    words = dict()
    
    for file in files:
        result = await get_training_data(browser, file)
        write_to_resource_target(file, result)
        words[file] = result
        
                
    await browser.close()

async def get_training_data(browser: ptr.browser.Browser, file_path: str) -> PosTagList:
    df = pd.read_csv(file_path)
    df.rename(columns=lambda x: x.strip(), inplace=True)

    words = []
    if isinstance(df, pd.DataFrame):
        for _, row in df.iterrows():
            print(row)
            result = await download_html(browser, row["link"], row["selector"])
            words.extend(clean_up_words(tokenize(result)))
    
    return words

In [6]:
## Cleaners.
def tokenize(data: str):
    tokenized_words = nltk.word_tokenize(data)
    tagged_words = nltk.pos_tag(tokenized_words)
    return tagged_words

def filter_words(x: PosTag, fns: list[Callable[[PosTag], bool]], keep=True) -> bool:
    if not keep:
        return False
    
    if fns:
        return filter_words(x, fns[1:], keep=fns[0](x))
    
    return True
    
def filter_by_duplicate(x: tuple[str, str]) -> bool:
    return x[0] != x[1]

def filter_by_stop_word(x: tuple[str, str]) -> bool:
    return x[0] not in stopwords.words("english")

def filter_by_alphabet(x: tuple[str, str]) -> bool:
    regex = re.compile("^([a-zA-Z]|')+$")
    return regex.match(x[0])
            
def clean_up_words(words: PosTagList) -> PosTagList:
    return list(filter(
        lambda x: filter_words(x, [filter_by_duplicate, filter_by_stop_word, filter_by_alphabet]),
        words
        )
    )

In [7]:
# type: ignore
await (get_training_data_from_folder("../resources/sources"))

selector                                           #topicText
link        https://www.uptodate.com/contents/depression-t...
Name: 0, dtype: object
selector    #main-content > div:nth-child(1) > div.content...
link        https://www.mayoclinic.org/diseases-conditions...
Name: 1, dtype: object
selector                                     #mw-content-text
link        https://simple.wikipedia.org/wiki/Depression_(...
Name: 2, dtype: object
selector                                         #maincontent
link        https://www.nhs.uk/mental-health/conditions/cl...
Name: 3, dtype: object
selector                   #overview > div > div > section
link        https://familydoctor.org/condition/depression/
Name: 4, dtype: object
selector    #__next > div.css-fdjy12 > div:nth-child(5) > ...
link        https://www.healthline.com/health/depression#s...
Name: 5, dtype: object
selector    #__next > div.css-fdjy12 > div:nth-child(5) > ...
link        https://www.medicalnewstoday.com/articles/3213...


FileNotFoundError: [Errno 2] No such file or directory: '../resources/targets/../resources/sources/depression.csv'